# Challenger Protocol: Dual-Model Forecasting Demo

**Phase 3 of Chrono_LLM_RAG_Enhanced**  
**Author:** Shohruh127  
**Date:** 2025-11-26

## Overview

This notebook demonstrates the Challenger Protocol - a dual-model forecasting approach that validates AI predictions against statistical baselines.

### Key Components:
1. **Chronos-2 Engine** - AI-based probabilistic forecasting
2. **ETS Challenger** - Statistical baseline (Exponential Smoothing)
3. **Uncertainty Flagger** - Automatic divergence detection

### Why This Matters:
- Prevents blind trust in AI predictions
- Identifies high-uncertainty forecasts requiring analyst review
- Provides economic reality check via statistical methods

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

# Import forecasting modules
from forecasting import ChronosEngine, ETSChallenger, UncertaintyFlagger, forecast_with_validation

print("✅ Imports successful!")

## 1. Sample Economic Data

Let's create a sample economic time series representing annual indicators (2016-2024).

In [ ]:
# Sample economic series (2016-2024) - e.g., Industry output index
years = pd.date_range('2016', '2025', freq='YS')[:9]
industry_output = pd.Series(
    [100, 105, 110, 108, 115, 120, 125, 130, 128],
    index=years,
    name='Industry Output Index'
)

# Display the data
print("Historical Data (2016-2024):")
print(industry_output)

# Plot historical data
plt.figure(figsize=(10, 5))
plt.plot(industry_output.index, industry_output.values, 'o-', linewidth=2, markersize=8)
plt.title('Historical Industry Output Index (2016-2024)', fontsize=14, fontweight='bold')
plt.xlabel('Year')
plt.ylabel('Index Value')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 2. Initialize Models

In [ ]:
# Initialize the three components
chronos = ChronosEngine()
ets = ETSChallenger()
flagger = UncertaintyFlagger(high_threshold=0.20, medium_threshold=0.10)

print("✅ Models initialized:")
print(f"   - Chronos Engine: {chronos.model_id}")
print(f"   - ETS Challenger: Auto-select={ets.auto_select}")
print(f"   - Uncertainty Flagger: High={flagger.high_threshold*100}%, Medium={flagger.medium_threshold*100}%")

## 3. Generate Forecasts

Forecast 2025 and 2026 using both models.

In [ ]:
# Forecast horizon: 2 years (2025, 2026)
horizon = 2

# Run Chronos-2 forecast
print("🔮 Running Chronos-2 forecast...")
chronos_result = chronos.forecast(industry_output, horizon=horizon)
print(f"   Chronos Predictions (2025, 2026): {chronos_result['median']}")

# Run ETS challenger forecast
print("\n📊 Running ETS challenger forecast...")
ets_result = ets.forecast(industry_output, horizon=horizon)
print(f"   ETS Predictions (2025, 2026): {ets_result['median']}")
print(f"   Model Type: {ets_result['model_type']}")

## 4. Compare and Flag Divergence

In [ ]:
# Compare forecasts
comparison = flagger.compare(chronos_result, ets_result)

# Display comparison
print("\n⚖️  Forecast Comparison:")
print("="*70)
for step, details in comparison.items():
    year = 2025 if step == 'step_1' else 2026
    print(f"\n{year}:")
    print(f"   Chronos: {details['chronos']:.2f}")
    print(f"   ETS:     {details['ets']:.2f}")
    print(f"   Divergence: {details['divergence_pct']:.2f}%")
    print(f"   Flag: {details['flag']}")
    if details['requires_review']:
        print(f"   ⚠️  REQUIRES ANALYST REVIEW")

# Get summary
summary = flagger.summary_report(comparison)
print("\n\n📈 Summary Statistics:")
print("="*70)
print(f"Total forecast points: {summary['total_points']}")
print(f"High uncertainty: {summary['high_uncertainty']}")
print(f"Medium uncertainty: {summary['medium_uncertainty']}")
print(f"Low uncertainty: {summary['low_uncertainty']}")
print(f"Require review: {summary['requires_review']}")
print(f"Average divergence: {summary['avg_divergence_pct']:.2f}%")
print(f"Max divergence: {summary['max_divergence_pct']:.2f}%")

## 5. Visualize Results

In [ ]:
# Prepare data for visualization
forecast_years = pd.date_range('2025', '2027', freq='YS')[:2]

# Create comprehensive visualization
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# Plot 1: Historical + Forecasts
ax1.plot(industry_output.index, industry_output.values, 'o-', 
         linewidth=2, markersize=8, label='Historical', color='blue')
ax1.plot(forecast_years, chronos_result['median'], 's--', 
         linewidth=2, markersize=8, label='Chronos-2', color='red')
ax1.plot(forecast_years, ets_result['median'], '^--', 
         linewidth=2, markersize=8, label='ETS', color='green')

# Add uncertainty bounds for Chronos
ax1.fill_between(forecast_years, 
                  chronos_result['quantile_10'], 
                  chronos_result['quantile_90'],
                  alpha=0.2, color='red', label='Chronos 80% CI')

ax1.axvline(x=industry_output.index[-1], color='gray', linestyle=':', alpha=0.5)
ax1.text(industry_output.index[-1], ax1.get_ylim()[1]*0.95, 
         'Forecast Start', ha='right', va='top', fontsize=9)

ax1.set_title('Dual-Model Forecast Comparison', fontsize=14, fontweight='bold')
ax1.set_xlabel('Year')
ax1.set_ylabel('Industry Output Index')
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)

# Plot 2: Divergence Analysis
divergences = [comparison['step_1']['divergence_pct'], comparison['step_2']['divergence_pct']]
flags = [comparison['step_1']['flag'], comparison['step_2']['flag']]
colors = ['green' if f == 'LOW' else 'orange' if f == 'MEDIUM' else 'red' for f in flags]

bars = ax2.bar(range(len(divergences)), divergences, color=colors, alpha=0.7)
ax2.axhline(y=10, color='orange', linestyle='--', linewidth=1, label='Medium Threshold (10%)')
ax2.axhline(y=20, color='red', linestyle='--', linewidth=2, label='High Threshold (20%)')

ax2.set_title('Forecast Divergence Analysis', fontsize=14, fontweight='bold')
ax2.set_xlabel('Forecast Year')
ax2.set_ylabel('Divergence (%)')
ax2.set_xticks(range(len(divergences)))
ax2.set_xticklabels(['2025', '2026'])
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for i, (bar, div, flag) in enumerate(zip(bars, divergences, flags)):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
             f'{div:.1f}%\n{flag}',
             ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

## 6. Convenience Function

The `forecast_with_validation` function combines all steps into one call.

In [ ]:
# Use the convenience function
result = forecast_with_validation(
    series=industry_output,
    horizon=2,
    uncertainty_threshold=0.20
)

print("📋 Forecast with Validation Results:")
print("="*70)
print(f"\nChronoS-2 Forecast: {result['chronos_forecast']['median']}")
print(f"ETS Forecast: {result['ets_forecast']['median']}")
print(f"\nRequires Review: {result['requires_review']}")

if result['requires_review']:
    print("\n⚠️  WARNING: High uncertainty detected!")
    print("Analyst review is recommended before making decisions.")
    flagged_points = flagger.get_flagged_points(result['comparison'])
    print(f"\nFlagged points: {len(flagged_points)}")
    for point in flagged_points:
        print(f"  - {point['step']}: Divergence {point['divergence_pct']:.1f}%")
else:
    print("\n✅ Low divergence - forecasts are aligned.")

## 7. Batch Forecasting Example

Demonstrate forecasting multiple time series at once.

In [ ]:
# Create multiple series
data_df = pd.DataFrame({
    'Year': years,
    'Industry': [100, 105, 110, 108, 115, 120, 125, 130, 128],
    'Agriculture': [200, 210, 215, 220, 225, 230, 235, 240, 238],
    'Services': [50, 52, 55, 58, 60, 62, 65, 68, 70]
})

data_df.set_index('Year', inplace=True)

print("Multiple Time Series:")
print(data_df)

# Batch forecast with Chronos
batch_results = chronos.forecast_batch(
    df=data_df,
    target_cols=['Industry', 'Agriculture', 'Services'],
    horizon=2
)

print("\nBatch Forecast Results:")
print(batch_results)

## 8. Generate Analyst Review Report

Create a formatted report for high-uncertainty forecasts.

In [ ]:
def generate_review_report(series_name: str, comparison: dict, chronos_result: dict, ets_result: dict):
    """
    Generate analyst review report for high-uncertainty forecasts.
    """
    report = []
    report.append("=" * 80)
    report.append("ANALYST REVIEW REPORT")
    report.append("=" * 80)
    report.append(f"\nTime Series: {series_name}")
    report.append(f"Date: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}")
    report.append(f"\nAI Model: {chronos_result.get('model_version', 'Chronos-2')}")
    report.append(f"Statistical Model: {ets_result.get('model_type', 'ETS')}")
    
    report.append("\n" + "-" * 80)
    report.append("FORECAST COMPARISON")
    report.append("-" * 80)
    
    for step, details in comparison.items():
        year = 2025 if step == 'step_1' else 2026
        report.append(f"\n{year}:")
        report.append(f"  Chronos-2 Prediction: {details['chronos']:.2f}")
        report.append(f"  ETS Prediction: {details['ets']:.2f}")
        report.append(f"  Divergence: {details['divergence_pct']:.2f}%")
        report.append(f"  Uncertainty Flag: {details['flag']}")
        
        if details['requires_review']:
            report.append(f"  ⚠️  STATUS: REQUIRES REVIEW")
            report.append(f"  ⚠️  REASON: Divergence exceeds 20% threshold")
    
    report.append("\n" + "-" * 80)
    report.append("RECOMMENDATIONS")
    report.append("-" * 80)
    
    high_flags = sum(1 for v in comparison.values() if v['flag'] == 'HIGH')
    if high_flags > 0:
        report.append("\n⚠️  HIGH UNCERTAINTY DETECTED")
        report.append("\nRecommended Actions:")
        report.append("1. Investigate underlying economic factors driving divergence")
        report.append("2. Review historical data for anomalies or structural breaks")
        report.append("3. Consider consulting domain experts")
        report.append("4. Use conservative estimates for planning")
    else:
        report.append("\n✅ Forecasts are aligned within acceptable thresholds.")
        report.append("\nBoth models agree - predictions can be used with confidence.")
    
    report.append("\n" + "=" * 80)
    
    return "\n".join(report)

# Generate report
report = generate_review_report(
    series_name="Industry Output Index",
    comparison=comparison,
    chronos_result=chronos_result,
    ets_result=ets_result
)

print(report)

## 9. Working with DataFrames

Demonstrate adding uncertainty flags to DataFrames.

In [ ]:
# Create forecast DataFrame
forecast_df = pd.DataFrame({
    'year': [2025, 2026],
    'chronos_forecast': chronos_result['median'],
    'ets_forecast': ets_result['median']
})

print("Original Forecast DataFrame:")
print(forecast_df)

# Add uncertainty flags
flagged_df = flagger.flag_dataframe(forecast_df)

print("\nFlagged Forecast DataFrame:")
print(flagged_df)

# Filter high-uncertainty points
high_uncertainty = flagged_df[flagged_df['_requires_review'] == True]
if len(high_uncertainty) > 0:
    print("\n⚠️  High Uncertainty Points:")
    print(high_uncertainty)
else:
    print("\n✅ No high-uncertainty points detected.")

## Summary

This notebook demonstrated the Challenger Protocol for validating AI forecasts:

### Key Takeaways:
1. **Dual-Model Approach**: Always validate AI with statistical baselines
2. **Automatic Flagging**: >20% divergence triggers analyst review
3. **Transparency**: Clear visibility into model disagreements
4. **Economic Reality Check**: ETS provides grounded comparison

### When to Use:
- ✅ High-stakes economic forecasts
- ✅ Limited historical data (< 20 points)
- ✅ Need for explainable AI
- ✅ Risk-averse decision making

### Next Steps:
1. Apply to your own economic data
2. Adjust thresholds based on domain requirements
3. Integrate with existing forecasting workflows
4. Build automated alerting for high-uncertainty forecasts